In [1]:
import ftplib
import gc
import glob
import os
import tarfile

import geopandas as gpd
import rasterio
import rioxarray as rio
from rasterio.features import geometry_mask
from rasterio.warp import Resampling, calculate_default_transform, reproject
from rioxarray.merge import merge_arrays

# Svalbard catchment delineation

# Notebook 01: Basic processing

**The merging and resampling steps (sections 3 and 4) require a large machine (240 GB memory).**

The [ArcticDEM website](https://www.pgc.umn.edu/data/arcticdem/) has 2 m and 10 m resolution elevation datasets covering most of the Arctic. The datasets are tiled and a tile index for each dataset is provided as a shapefile with a download URL in the attribute table. I have downloaded the tile indexes and deleted all tiles except for those covering Svalbard. These are stored in the folders below:

    shared/common/01_datasets/spatial/svalbard/arctic_dem_10m/ArcticDEM_Mosaic_Index_v4_1_10m_SvalbardOnly.shp
    
    shared/common/01_datasets/spatial/svalbard/arctic_dem_2m/ArcticDEM_Mosaic_Index_v4_1_2m_SvalbardOnly.shp

This notebook downloads the 10 m raw data, reprojects it to UTM Zone 33 N, and merges the datasets to a single GeoTIFF. Downsampled versions at 20 m and 40 m are also created. The notebook can also be used to download and merge the 2 m dataset, but the output file will likely be too cumbersome to be used effectively (~50 GB compressed?). To work at 2 m resolution, we probably need to split the data into "main catchments", like I did for the national catchment tool.

In [2]:
# Dataset resolution to download (either 2 m or 10 m)
res = 10

# Projected CRS to use (EPSG)
epsg = 25833

# FTP server details. Faster and more reliable than downloading over http.
# See https://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/
ftp_server = "ftp.data.pgc.umn.edu"

# Properties for output mosaic
no_data_val = -9999
dst_dtype = "float32"  # Rasterio dtypes: https://test2.biogeo.ucdavis.edu/rasterio/_modules/rasterio/dtypes.html
bbox = (400000, 8480000, 754000, 8980000)  # xmin, ymin, xmax, ymax

# Polygons representing sea for masking strange values
sea_mask_shp = (
    r"/home/jovyan/shared/common/01_datasets/spatial/svalbard/vector/svalbard_sea.shp"
)

In [3]:
# Validate user options
assert res in (2, 10), "'res' must be either 2 or 10 (metres)."
assert isinstance(epsg, int), "EPSG code must be an integer."

# Build folder paths
base_dir = f"/home/jovyan/shared/common/01_datasets/spatial/svalbard/arctic_dem_{res}m"
raw_dir = os.path.join(base_dir, "raw_tiles")
proj_dir = os.path.join(base_dir, "proj_tiles")
merge_dir = os.path.join(base_dir, "merged")

# Build folders
for dir_path in [base_dir, raw_dir, proj_dir, merge_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

## 1. Data download

In [4]:
%%time

# Read tile index
idx_shp_path = os.path.join(
    base_dir, f"ArcticDEM_Mosaic_Index_v4_1_{res}m_SvalbardOnly.shp"
)
idx_gdf = gpd.read_file(idx_shp_path)
print(len(idx_gdf), "DEM tiles to download.\n")

# Connect to FTP server
ftp = ftplib.FTP(ftp_server)
ftp.login()

# Loop over tiles
for idx, row in idx_gdf.iterrows():
    url = row["fileurl"].split("data.pgc.umn.edu")[1]
    fname = os.path.basename(url)
    fpath = os.path.join(raw_dir, fname)
    print(f"Processing {fname}:")

    # Download tile via FTP
    print("  Downloading...")
    with open(fpath, "wb") as f:
        ftp.retrbinary(f"RETR {url}", f.write)

    # Unzip, keeping only specific files
    print("  Unzipping...")
    with tarfile.open(fpath, "r:gz") as tar_ref:
        for member in tar_ref.getmembers():
            if any(
                member.name.endswith(suffix)
                for suffix in ["_browse.tif", "_datamask.tif", "_dem.tif"]
            ):
                tar_ref.extract(member, raw_dir)

ftp.quit()

/opt/conda/lib/python3.11/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)


17 DEM tiles to download.

Processing 34_50_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 37_51_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 35_53_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 35_52_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 33_53_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 35_51_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 33_51_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 37_50_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 36_50_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 36_53_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 33_52_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 36_52_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 34_53_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 34_51_10m_v4.1.tar.gz:
  Downloading...
  Unzipping...
Processing 36_51_10m_v4.1.tar.gz:
  Downloading..

'221 Goodbye.'

## 2. Mask raw tiles

Some of the DEM tiles have strange values offshore (e.g. areas of the sea at > 30 m above sea level). This causes issues during terrain processing, because the pit-filling algorithm ends up filling river valleys up to around 30 m in order to flow over this weird offshore "sea wall". However, the raw data for each tile also includes a data mask, which can be used to set values offshore to NoData. This is done below before reprojecting and merging.

In [5]:
%%time

# List of DEM files to process
search_path = os.path.join(raw_dir, "*_dem.tif")
flist = glob.glob(search_path)

for dem_path in flist:
    # Get paths to DEM tile and corresponding data mask
    dem_name = os.path.basename(dem_path)
    mask_name = dem_name.replace("_dem.tif", "_datamask.tif")
    mask_path = os.path.join(raw_dir, mask_name)

    # Read data to arrays
    with rasterio.open(dem_path) as dem_src:
        dem_data = dem_src.read(1)
        dem_meta = dem_src.meta

    with rasterio.open(mask_path) as mask_src:
        mask_data = mask_src.read(1)

    # Set values in DEM to NoData where mask is zero
    dem_data[mask_data == 0] = dem_meta["nodata"]

    # Save
    masked_dem_path = os.path.join(
        raw_dir, dem_name.replace("_dem.tif", "_dem-masked.tif")
    )
    with rasterio.open(masked_dem_path, "w", **dem_meta) as dest:
        dest.write(dem_data, 1)

CPU times: user 33.7 s, sys: 10.6 s, total: 44.4 s
Wall time: 1min 14s


## 3. Reproject

In [6]:
def reproject_file(src_path, dst_path, dst_crs, resolution):
    """Reproject 'src_path' to 'dst_crs' with a resolution of 'resolution'
    and save to 'dst_path'.

    Args
        src_path: Str. Path to source dataset (GeoTiff).
        dst_path: Str. Dataset to create (GeoTiff).
        dst_crs: Str. EPSG code for final dataset. Format: 'epsg:XXXX'.
        resolution: Int. Cell size for output in units of 'dst_crs'.

    Returns
        None. 'dst_path' is created.
    """
    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds, resolution=resolution
        )
        kwargs = src.meta.copy()
        kwargs.update(
            {
                "crs": dst_crs,
                "transform": transform,
                "width": width,
                "height": height,
                "compress": "lzw",
                "dtype": "float32",
            }
        )

        with rasterio.open(dst_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.bilinear,
                )

In [7]:
%%time

# List of DEM files to reproject
search_path = os.path.join(raw_dir, "*_dem-masked.tif")
flist = glob.glob(search_path)

# Reproject
for src_path in flist:
    src_fname = os.path.basename(src_path)
    dst_fname = src_fname.replace("_dem-masked.tif", "_dem_proj.tif")
    dst_path = os.path.join(proj_dir, dst_fname)
    reproject_file(src_path, dst_path, f"EPSG:{epsg}", res)

CPU times: user 5min 8s, sys: 13.6 s, total: 5min 21s
Wall time: 5min 37s


## 4. Merge

In [8]:
%%time

# List of DEM files to merge
search_path = os.path.join(proj_dir, "*_dem_proj.tif")
flist = glob.glob(search_path)

print("Opening files...")
srcs = [rio.open_rasterio(fpath, mask_and_scale=True, cache=False) for fpath in flist]

print("Merging tiles...")
rds = merge_arrays(srcs, bounds=bbox, res=res)

print("Saving...")
merge_path = os.path.join(merge_dir, f"svalbard_{res}m_dem_proj.tif")
rds.rio.write_nodata(no_data_val, inplace=True)
rds.rio.to_raster(
    merge_path,
    compress="lzw",
    BIGTIFF="YES",
    tiled=True,
    dtype=dst_dtype,
)
srcs = [src.close() for src in srcs]
rds.close()
del srcs, rds
gc.collect()

print("Done.")

Opening files...
Merging tiles...
Saving...
Done.
CPU times: user 3min 55s, sys: 1min 45s, total: 5min 40s
Wall time: 5min 41s


## 5. Set sea mask

The coastline in the raster data does not match that in the vector data. This leads to issues where vector streams stop before they reach the raster coast, which causes issues with the "burning". This codes uses the vector coastline to set cells in the sea to NaN.

In [9]:
# Read merged DEM
with rasterio.open(merge_path) as src:
    meta = src.meta.copy()
    data = src.read(1)

# Read sea polys and convert to mask
gdf = gpd.read_file(sea_mask_shp)
mask = geometry_mask(
    [geom for geom in gdf.geometry],
    transform=src.transform,
    invert=True,
    out_shape=src.shape,
)

# Set sea to NoData
data[mask] = meta["nodata"]

# Save
with rasterio.open(merge_path, "w", **meta) as dst:
    dst.write(data, 1)

## 6. Resampling

In [10]:
%%time

print("Downsampling to 20m...")

# Build folders
base_dir_20m = r"/home/jovyan/shared/common/01_datasets/spatial/svalbard/arctic_dem_20m"
merge_dir_20m = os.path.join(base_dir_20m, "merged")
for dir_path in [base_dir_20m, merge_dir_20m]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Resample
merged_10m_path = os.path.join(merge_dir, r"svalbard_10m_dem_proj.tif")
rds = rio.open_rasterio(merged_10m_path, mask_and_scale=True, cache=False)
upscale_factor = 0.5
width = int(rds.rio.width * upscale_factor)
height = int(rds.rio.height * upscale_factor)
rds = rds.rio.reproject(
    rds.rio.crs,
    shape=(height, width),
    resampling=Resampling.bilinear,
)

# Save
merged_20m_path = os.path.join(merge_dir_20m, r"svalbard_20m_dem_proj.tif")
rds.rio.to_raster(
    merged_20m_path, compress="lzw", BIGTIFF="IF_SAFER", tiled=True, dtype=dst_dtype
)
rds.close()
del rds
gc.collect()

Downsampling to 20m...
CPU times: user 1min 28s, sys: 19.6 s, total: 1min 48s
Wall time: 1min 58s


99

In [11]:
%%time

print("Downsampling to 40m...")

# Build folders
base_dir_40m = r"/home/jovyan/shared/common/01_datasets/spatial/svalbard/arctic_dem_40m"
merge_dir_40m = os.path.join(base_dir_40m, "merged")
for dir_path in [base_dir_40m, merge_dir_40m]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Resample
merged_10m_path = os.path.join(merge_dir, r"svalbard_10m_dem_proj.tif")
rds = rio.open_rasterio(merged_10m_path, mask_and_scale=True, cache=False)
upscale_factor = 0.25
width = int(rds.rio.width * upscale_factor)
height = int(rds.rio.height * upscale_factor)
rds = rds.rio.reproject(
    rds.rio.crs,
    shape=(height, width),
    resampling=Resampling.bilinear,
)

# Save
merged_40m_path = os.path.join(merge_dir_40m, r"svalbard_40m_dem_proj.tif")
rds.rio.to_raster(
    merged_40m_path, compress="lzw", BIGTIFF="IF_SAFER", tiled=True, dtype=dst_dtype
)
rds.close()
del rds
gc.collect()

Downsampling to 40m...
CPU times: user 51.8 s, sys: 15.4 s, total: 1min 7s
Wall time: 1min 7s


98